In [1]:
%matplotlib inline
import tkinter as tk
from tkinter import messagebox, filedialog, Canvas, Entry, Button, PhotoImage
from rdkit.Chem import rdDepictor
import sys
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
from datetime import datetime


from Chem_pack.smiles_to_molar_mass import smiles_to_molar_mass
from Chem_pack.name_to_smiles import name_to_smiles
from Chem_pack.display_molecule import display_molecule
from Chem_pack.make_graph import make_graph
from Chem_pack.linear_regression import linear_regression
from Chem_pack.error_calculation_interface import error_calculation_interface
from Chem_pack.relative_to_assets import relative_to_assets


sys.path.insert(0, "../src")
rdDepictor.SetPreferCoordGen(True)
set_matplotlib_formats('png', 'pdf')
plt.rcParams['font.family'] = 'Times New Roman'




In [2]:

entry_input = None
selected_radio = None
window = None
graph_window = None
mol_window = None
x_label = None
y_label = None
title = None
result_text = None



def process_input(event=None):
    global selected_radio, x_label, y_label, title, result_text
    if selected_radio.get() == "1":
        smiles_molecule = name_to_smiles(entry_input.get().strip())
        if smiles_molecule is not None:
            result_text = f"The SMILES for the molecule '{entry_input.get().strip()}' is: {smiles_molecule}"
        else:
            result_text = f"The molecule '{entry_input.get().strip()}' was not found in the PubChem database."
    elif selected_radio.get() == "2":
        molar_mass = smiles_to_molar_mass(entry_input.get().strip())
        if molar_mass is not None:
            result_text = f"The molar mass of the molecule is: {molar_mass} g/mol"
        else:
            result_text = "Invalid SMILES or molecule not found."
    elif selected_radio.get() in ["3", "4"]:
        if selected_radio.get() == "3":
            make_graph(entry_input, x_label, y_label, title)
        else:
            linear_regression(entry_input, x_label, y_label, title)
        return  # Return to prevent displaying the result window
    elif selected_radio.get() == "5":  # Ajouter cette condition pour le bouton radio "5"
        display_molecule(entry_input)  # Appeler la fonction pour afficher la structure moléculaire
        return  # Return to prevent displaying the result window

    elif selected_radio.get() == "6":
        error_calculation_interface()
    else:
        result_text = "Please select an input type."

    display_result(result_text)



def display_result(result_text):
    result_window = tk.Toplevel(window)
    result_window.title("Result")


    result_textbox = tk.Text(result_window, wrap="word", font=("Times New Roman", 25), fg="#BBE1FA", bg = "#1B262C", height=10, width=30)
    result_textbox.insert("1.0", result_text)
    result_textbox.config(state="disabled")
    result_textbox.grid(row=0, column=0, padx=0, pady=0)


def browse_excel_file():
    global entry_input
    filepath = filedialog.askopenfilename(title="Select Excel File", filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*")))
    if filepath:
        try:
            entry_input.delete(0, tk.END)  
            entry_input.insert(0, filepath)  
            print("Selected Excel file:", filepath)
            print("File path copied to clipboard.")
        except Exception as e:
            print("Error:", e)
def select_all(event):
    if window and window.winfo_exists():
        event.widget.tag_add("sel", "1.0", "end")
    return "break"

def copy_text(event):
    try:
        if window and window.winfo_exists():
            if event and event.widget and event.widget.winfo_exists():
                event.widget.event_generate("<<Copy>>")
    except Exception as e:
        print("Error while copying text:", e)
    return "break"

def bind_enter(event):
    try:
        if event.keysym == "Return":
            process_input()
    except Exception as e:
        print("Error while binding Enter key:", e)


if window:
    window.bind("<KeyPress>", bind_enter)





In [3]:

window = tk.Tk()  

x_axis_label = tk.StringVar()
y_axis_label = tk.StringVar()
graph_title = tk.StringVar()

window.title("Project Tools-Kit")
input_type = tk.IntVar() #creates a Tkinter IntVar variable, which is used to track the value of the selected input type. In this case, it's initialized to 1
input_type.set(1)

def welcome_message():
    global entry_input
    welcome_window = tk.Toplevel(window)
    welcome_window.title("Welcome Message")

    welcome_text = (
        "✨ Welcome to Our Project! ✨\n\n"
        "We are excited to share our project with you.\n\n"
        "Discover all the details on our GitHub repository:\n"
        "🔗 [Project ppchem Tools Kit] (https://github.com/sgrunber/Project-ppchem-tools-kit)\n\n"
        "Happy exploring! 🚀\n\n"
    )

    welcome_textbox = tk.Text(welcome_window, wrap="word", font=("Times New Roman", 20), fg="#BBE1FA", bg = "#1B262C", height=10, width=50)
    welcome_textbox.insert("1.0", welcome_text)
    welcome_textbox.config(state="disabled")
    welcome_textbox.tag_configure("center", justify="center")
    welcome_textbox.tag_add("center", "1.0", "end")
    welcome_textbox.grid(row=0, column=0, padx=0, pady=0)


window.geometry("1000x800")
window.configure(bg = "#1B262C")

canvas = Canvas(window, bg="#1B262C", height=800, width=1000, bd=0, highlightthickness=0, relief="ridge")
canvas.place(x=0, y=0)

canvas.create_text(12.0, 194.0, anchor="nw", text="Choose Input Type :", fill="#BBE1FA", font=("Times New Roman", 27, "bold"))


canvas.create_text(92.0, 378.0, anchor="nw",text="Input :", fill="#BBE1FA", font=("Times New Roman", 30, "bold"))
entry_input = Entry(bd=1, bg="#0F4C75", fg="#BBE1FA", highlightthickness=0, font=("Times New Roman", 25))
entry_input.place(x=272.5, y=368.0, width=605.0, height=74.0)

button_image_browse = PhotoImage(file=relative_to_assets("button_browse.png")) 
button_browse = Button(image=button_image_browse, borderwidth=0, highlightthickness=0, command=browse_excel_file, relief="flat")
button_browse.place(x=409.0, y=500.0, width=154.2904052734375, height=60.0)

button_image_process = PhotoImage(file=relative_to_assets("button_process.png"))
button_process = Button(image=button_image_process, borderwidth=0, highlightthickness=0, command=process_input, relief="flat")
button_process.place(x=394.5, y=620.0, width=182.0, height=73.0)

button_image_title = PhotoImage(file=relative_to_assets("button_title.png"))
button_title = Button(image=button_image_title, borderwidth=0, highlightthickness=0, command=welcome_message, relief="flat")
button_title.place(x=273.0, y=36.0, width=455.0, height=90.0)



logo_image = PhotoImage(file=relative_to_assets('EPFL_logo.png'))
logo_label = tk.Label(window, image=logo_image)
logo_label.place(x=10.0, y=10.0, width=150.0, height=60.0)

clock_label = tk.Label(window, text="", font=("Times New Roman", 20), bg="#1B262C", fg="#BBE1FA")
clock_label.place(x=900, y=10)



def clear_input(): 
    entry_input.delete(0, tk.END)
    
def on_radio_select(value):
    selected_radio.set(value)
    clear_input()

def create_radio_button(x, y, text, value):  
    radio_button = tk.Radiobutton(canvas, text=text, variable=selected_radio, value=value,
                                  command=lambda: on_radio_select(value),
                                  font=("Times New Roman", 20), bg="#1B262C")
    canvas.create_window(x, y, anchor="nw", window=radio_button)

radio_button_data = [
    (291, 194, "Molecule Name", "1"),
    (291, 252, "Excel Graph", "3"),
    (700, 194, "Linear Regression", "4"), 
    (700, 252, "Show Molecule", "5"),
    (469, 252, "Error calculation", "6"),
    (469, 194, "Molecular weight", "2")
]  

selected_radio = tk.StringVar(value="none")
for data in radio_button_data:
    create_radio_button(*data)


#canvas.bind('<Return>', bind_enter)

def update_clock():
    current_time = datetime.now().strftime('%H:%M:%S')
    current_date = datetime.now().strftime('%Y-%m-%d')
    clock_label.config(text=current_time)
    date_label.config(text=current_date)
    window.after(1000, update_clock)

date_label = tk.Label(window, text="", font=("Times New Roman", 20), bg="#1B262C", fg="#FFFFFF")
date_label.place(x=890, y=10)

clock_label = tk.Label(window, text="", font=("Times New Roman", 20), bg="#1B262C", fg="#FFFFFF")
clock_label.place(x=890, y=40)

update_clock()

window.unbind_all("<Return>")

def on_closing():
    global window
    if messagebox.askokcancel("Quit", "Are you sure you want to quit?"):
        window.destroy()
window.protocol("WM_DELETE_WINDOW", on_closing)

window.resizable(False, False)

window.bind("<Return>", process_input)

window.mainloop()
